In [1]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pathlib import Path
import sys
import os
os.chdir('../..')
sys.path.append('../..')

import menpo.io as mio
from menpo.landmark import labeller, left_ventricle_34,left_ventricle_34_trimesh,left_ventricle_34_trimesh1

from menpo.transform import ThinPlateSplines
from menpo.feature import *
from menpofit.aam import LucasKanadeAAMFitter, WibergInverseCompositional
from menpofit.fitter import noisy_shape_from_bounding_box
from menpofit.visualize import plot_cumulative_error_distribution
from menpowidgets import *
from menpodetect import load_dlib_left_ventricle_detector
from menpo.io import export_pickle,import_pickle
from menpo.visualize import print_progress
import menpo.visualize

import project.utils.labeller_lv as labels
from project.utils import tfrecords
import predict_lv as predict
import project.utils.visualisation as visualisation

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import scipy.io as sio
import skimage.io as io
from io import BytesIO
from IPython.display import Image
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist

import tensorflow as tf

# method to load a database
def load_database(path_to_images, crop_percentage,max_diagonal=400, max_images=None):
    images = []
    # load landmarked images
    for i in mio.import_images(path_to_images, max_images=max_images, verbose=True):
           
        # convert it to grayscale if needed
        if i.n_channels == 3:
            i = i.as_greyscale(mode='luminosity')
        # crop image
        i = i.crop_to_landmarks_proportion(crop_percentage)
        
        d = i.diagonal()
        if d > max_diagonal:
            i = i.rescale(float(max_diagonal) / d)
        # define a TriMesh which will be useful for Piecewise Affine Warp of HolisticAAM
        
        labeller(i, 'PTS', left_ventricle_34)#lv_34_trimesh
        labeller(i, 'PTS', left_ventricle_34_trimesh1)
        labeller(i, 'PTS', left_ventricle_34_trimesh)
        #i= i.resize([256, 256])
        # append it to the list
        images.append(i)
    return images

In [3]:
# hg 特征
from project.hourglass import estimator
from tensorflow.contrib import learn
from tensorflow.core.protobuf import config_pb2
from menpo.feature import *
import project.hourglass.params as hgparams
from tensorflow.python.estimator.inputs import numpy_io 

from tensorflow.python.training import coordinator
from tensorflow.python.training import queue_runner_impl

from skimage import transform 
params = {
    hgparams.N_FEATURES: 128,
    hgparams.N_HOURGLASS: 1,
    hgparams.N_RESIDUALS: 3,
}
# Where is the model located?
model_dir = Path('models/lv/lv_1hg_lr1e-3_decay10/')
params[hgparams.N_LANDMARKS] = 34
# Instantiate Estimator
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
run_config = tf.contrib.learn.RunConfig(
    gpu_memory_fraction=1, session_config=config)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
    with tf.device("/gpu:0"):
        nn = learn.Estimator(
        model_dir=str(model_dir),
        params=params,
        config=run_config,
        model_fn=estimator._model_fn)
        
        sess.run(tf.global_variables_initializer())
@ndfeature
def hg(image):
    _,h,w= image.shape
    image = image.reshape([w,h])
    image=transform.resize(image, (256, 256))
    image = image.reshape([1,256,256])
    data = np.zeros((1, 256, 256, 3), np.float32)  
    data[:,:, :, 0] = image
    data[:,:, :, 1] = image
    data[:,:, :, 2] = image
    data=data.astype(np.float32)
    #image=image/255.0
    with tf.device("/gpu:0"):
        input_fn = tf.estimator.inputs.numpy_input_fn(
          x={"image": data},
          num_epochs=1,
          shuffle=False)
        predictions = nn.predict(input_fn=input_fn)
        images_generator = visualisation.lv_predictions(predictions,
                                                    show_input_images=False,
                                                    show_combined_heatmap=True,
                                                    show_individual_heatmaps=False)

    images = menpo.base.LazyList.init_from_iterable(images_generator)

    def flatten(list_of_lists): return [
        item for sublist in list_of_lists for item in sublist]
    images = flatten(images)
    images[0] =images[0].resize([w, h]).rescale_pixels(0, 255.0)

    tf.reset_default_graph()
    return images[0].pixels
def hog_closure(image):
    return hog(image,cell_size=4, window_step_horizontal=1, window_step_vertical=1)

#hg(train_images[1])
#type(train_images[1])
#hg(train_images[4])
#visualize_images(hg(train_images[7]))

INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001DF28CD8240>, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_keep_checkpoint_every_n_hours': 10000, '_evaluation_master': '', '_model_dir': 'models\\lv\\lv_1hg_lr1e-3_decay10', '_task_id': 0, '_environment': 'local', '_task_type': None, '_save_summary_steps': 100, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_session_config': gpu_options {
  allow_growth: true
}
, '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_master': ''}


## Training Parameters

In [4]:
max_images = 242
path_to_images = 'k:/datasets/lv_pts_216/trainset' #/media/taopan/data/
train_images = load_database(path_to_images,crop_percentage=0.5,max_images=max_images)

Found 212 assets, index the returned LazyList to import.


In [5]:
group = 'lv_34'

features = no_op
patch_normalisation = normalize_norm
patch_shape = (17, 17)
diagonal = None
scales = (1,0.5)
offsets = np.meshgrid(range(-4, 5, 2), range(-4, 5, 2))
offsets = np.asarray([offsets[0].flatten(), offsets[1].flatten()]).T 
#offsets = None
covariance = 2
max_shape_components = 25
max_appearance_components = 250
@imgfeature
def custom_double_igo(image):
    return igo(igo(image))
#dsift(training_images[0] ).view()
@ndfeature
def float32_fast_dsift(x):
    return fast_dsift(x).as_type(np.float32)

In [6]:
from menpofit.unified_aam_clm import UnifiedAAMCLM

unified_path = Path('I:/menpo/project_lv/data/deformable/unified/no-op-unified1.pkl')

if unified_path.exists():
    unified = mio.import_pickle(unified_path)
    print('load done')
else:
    unified  = UnifiedAAMCLM(train_images, group='lv_34', reference_shape=None,
                  diagonal=None, scales=(0.25, 0.5, 1.0), holistic_features=features, verbose=True,
                   max_appearance_components=0.9, max_shape_components=20)
    mio.export_pickle(unified,unified_path, overwrite=True, protocol=2)
    print('train done')
#print(unified)

- Normalizing images size: [          ] 6% (14/212) - 00:00:01 remaining        

i:\develop\menpo-py3\menpo\shape\pointcloud.py:261: MenpoDeprecationWarning: The .lms property is deprecated. LandmarkGroups are now shapes themselves - so you can use them directly anywhere you previously used .lms.Simply remove ".lms" from your code and things will work as expected (and this warning will go away)
  MenpoDeprecationWarning)


- Building modelsges size: [==========] 100% (212/212) - done.                  
  - Scale 0: Building appearance model                                          

i:\develop\menpofit-py3\menpofit\builder.py:338: MenpoFitModelBuilderWarning: The reference shape passed is not a TriMesh or subclass and therefore the reference frame (mask) will be calculated via a Delaunay triangulation. This may cause small triangles and thus suboptimal warps.
  MenpoFitModelBuilderWarning)


  - Scale 0: Donening experts: [==========] 100% (34/34) - done.                
  - Scale 1: Donening experts: [==========] 100% (34/34) - done.                
  - Scale 2: Donening experts: [==========] 100% (34/34) - done.                
                                                              train done


In [7]:
from menpo.image import Image
#Image(np.real(np.fft.fftshift(np.fft.ifft2(clm.classifiers[0][36].f)))).view()
#clm.view_expert_ensemble_widget()
#clm.view_shape_models_widget()
#clm.view_clm_widget()
#unified.parts_filters()[0][18].view()
#Image(unified.appearance_models[0].mean().pixels[18, 0]).view()